<a href="https://colab.research.google.com/github/D-kn/Big-Data-Projects/blob/main/4_pyspark_DataFrame_GroupBy_and_Aggregate_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 35.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=97d11f53fecf90215f4b23c73de21d976ec7a67057bbe206052ac4d11d7528f1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# mount drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/Projets Codes/Datasets'

/content/drive/My Drive/Projets Codes/Datasets


In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, avg, min, max

spark=SparkSession.builder.appName('Aggregate').getOrCreate()

In [5]:
df=spark.read.csv("Titanic.csv", header=True, inferSchema=True)

In [12]:
df.show(truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)           |female|35.0|1  

In [13]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [18]:
# Groupby Operations
df.groupBy("Sex").count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [17]:
# Groupby Operations
df.groupBy("Sex").sum("Age").show(truncate=False)

+------+--------+
|Sex   |sum(Age)|
+------+--------+
|female|7286.0  |
|male  |13919.17|
+------+--------+



In [20]:
df.groupBy("Sex").max("Age").show()

+------+--------+
|   Sex|max(Age)|
+------+--------+
|female|    63.0|
|  male|    80.0|
+------+--------+



In [45]:
df.groupBy("Sex").mean("Age").show()
# df.drop("PassengerId").groupBy("Survived", "Sex", "Pclass").mean().show(truncate=False)

+------+------------------+
|   Sex|          avg(Age)|
+------+------------------+
|female|27.915708812260537|
|  male| 30.72664459161148|
+------+------------------+



In [25]:
  # GroupBy on multiple columns
df.groupBy("Sex", "Pclass") \
  .sum("Age", "Fare") \
  .show(truncate=False)

+------+------+--------+------------------+
|Sex   |Pclass|sum(Age)|sum(Fare)         |
+------+------+--------+------------------+
|male  |3     |6706.42 |4393.586500000005 |
|female|3     |2218.5  |2321.1086000000005|
|female|1     |2942.0  |9975.824999999999 |
|female|2     |2125.5  |1669.7292         |
|male  |2     |3043.33 |2132.1125         |
|male  |1     |4169.42 |8201.587500000001 |
+------+------+--------+------------------+



In [31]:
# More aggregate functions

df.groupBy("Sex", "Pclass") \
  .agg(avg("Age").alias("avg_age"), \
       max("Age").alias("max_age"), \
       sum("Fare").alias("sum_fare"), \
       avg("Fare").alias("avg_Fare")
  ) \
  .show(truncate=False)

+------+------+------------------+-------+------------------+------------------+
|Sex   |Pclass|avg_age           |max_age|sum_fare          |avg_Fare          |
+------+------+------------------+-------+------------------+------------------+
|male  |3     |26.507588932806325|74.0   |4393.586500000005 |12.661632564841513|
|female|3     |21.75             |63.0   |2321.1086000000005|16.118809722222224|
|female|1     |34.61176470588235 |63.0   |9975.824999999999 |106.12579787234041|
|female|2     |28.722972972972972|57.0   |1669.7292         |21.97012105263158 |
|male  |2     |30.74070707070707 |70.0   |2132.1125         |19.74178240740741 |
|male  |1     |41.28138613861386 |80.0   |8201.587500000001 |67.22612704918033 |
+------+------+------------------+-------+------------------+------------------+



In [37]:
# Using aggregate with filter

df.groupBy("Sex", "Pclass") \
  .agg(avg("Age").alias("avg_age"), \
       max("Age").alias("max_age"), \
       sum("Fare").alias("sum_fare"), \
       avg("Fare").alias("avg_fare")) \
  .where(col("avg_fare") >= 16) \
  .show(truncate=False)

+------+------+------------------+-------+------------------+------------------+
|Sex   |Pclass|avg_age           |max_age|sum_fare          |avg_fare          |
+------+------+------------------+-------+------------------+------------------+
|female|3     |21.75             |63.0   |2321.1086000000005|16.118809722222224|
|female|1     |34.61176470588235 |63.0   |9975.824999999999 |106.12579787234041|
|female|2     |28.722972972972972|57.0   |1669.7292         |21.97012105263158 |
|male  |2     |30.74070707070707 |70.0   |2132.1125         |19.74178240740741 |
|male  |1     |41.28138613861386 |80.0   |8201.587500000001 |67.22612704918033 |
+------+------+------------------+-------+------------------+------------------+

